# 第八章：避免幻觉

- [课程](#lesson)

## 设置

运行以下设置单元格来加载您的API密钥并建立`get_completion`辅助函数。

In [ ]:
# 安装OpenAI库
%pip install openai==1.61.0

# 🔧 OpenAI环境自动配置
# 此设置会自动从环境变量或IPython存储中加载配置

# 安装OpenAI库
%pip install openai==1.61.0

# 导入Python内置的正则表达式库
import re

# 🚀 使用统一配置管理系统
from config import setup_notebook_environment, print_config_info

# 自动设置OpenAI客户端和get_completion函数
# 优先级：环境变量 > IPython存储 > 默认值
try:
    client, get_completion = setup_notebook_environment()
    print("✅ 使用统一配置管理成功！")
except Exception as e:
    print(f"❌ 统一配置失败，回退到传统方式: {e}")
    
    # 回退到传统的配置方式
    import openai
    
    # 从IPython存储中检索API_KEY和MODEL_NAME变量
    %store -r API_KEY
    %store -r MODEL_NAME

    # 如果没有设置MODEL_NAME，使用默认值
    try:
        MODEL_NAME
    except NameError:
        MODEL_NAME = "gpt-4o"  # 默认使用gpt-4o模型

    # 创建OpenAI客户端
    client = openai.OpenAI(api_key=API_KEY)

    def get_completion(prompt: str, system_prompt=""):
        """
        获取GPT的完成响应
        
        参数:
            prompt (str): 用户提示
            system_prompt (str): 系统提示（可选）
        
        返回:
            str: GPT的响应文本
        """
        # 构建消息列表
        messages = []
        
        # 如果有系统提示，添加系统消息
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        
        # 添加用户消息
        messages.append({"role": "user", "content": prompt})
        
        # 调用OpenAI API
        response = client.chat.completions.create(
            model=MODEL_NAME,              # 模型名称 (gpt-4o 或 deepseek-r1)
            messages=messages,             # 消息列表
            max_completion_tokens=2000,    # 最大token数
            temperature=0.0               # 温度参数，0表示更确定性
        )
        return response.choices[0].message.content
    
    print("⚠️  使用传统配置方式，建议配置环境变量以获得更好体验")


## 课程 

### 什么是LLM幻觉？

**LLM幻觉**（AI Hallucination）是指大语言模型生成看似合理但实际上不准确、虚假或无法验证的信息的现象。这是当前AI技术面临的一个重要挑战。

### 幻觉产生的原因

1. **训练数据限制**：模型基于有限的训练数据进行推理
2. **概率性生成**：模型基于概率分布生成文本，可能产生不存在的"合理"组合
3. **知识边界模糊**：模型难以准确识别自己的知识边界
4. **上下文理解偏差**：对复杂问题的理解可能存在偏差

### 避免幻觉的核心策略

**好消息是：有多种技术可以显著减少幻觉现象**。本节将重点介绍以下策略：

1. **🛡️ 承认无知策略**：给模型选择说"不知道"的机会
2. **🔍 证据导向策略**：要求模型先寻找证据再回答
3. **📝 分步推理策略**：引导模型逐步分析问题
4. **✅ 可信度评估**：要求模型评估自己答案的可信度

### 为什么避免幻觉很重要？

- **增强用户信任**：准确的信息建立用户对AI系统的信任
- **降低误导风险**：避免传播错误信息
- **提高实用性**：确保AI助手在实际应用中的可靠性
- **符合伦理要求**：负责任的AI开发实践

下面我们通过具体示例来学习这些技术的实际应用。

In [ ]:
# 📌 错误示例：没有给模型"不知道"的选项
bad_prompt = """
请告诉我关于 XYZ-9000 量子处理器的技术规格和性能参数。
"""

print("❌ 容易产生幻觉的提示:")
print(bad_prompt)
print("=" * 50)

response_bad = get_completion(bad_prompt)
print("模型回答:")
print(response_bad)


In [ ]:
# ✅ 正确示例：明确给出"不知道"的选项
good_prompt = """
请告诉我关于 XYZ-9000 量子处理器的技术规格和性能参数。

重要提示：
- 如果你对这个产品不确定或没有可靠信息，请直接说"我不知道"或"我没有这个产品的确切信息"
- 不要编造或猜测技术规格
- 只提供你确信准确的信息
"""

print("✅ 避免幻觉的提示:")
print(good_prompt)
print("=" * 50)

response_good = get_completion(good_prompt)
print("模型回答:")
print(response_good)


In [ ]:
# 📌 错误示例：直接要求答案，没有要求证据分析
direct_prompt = """
中国明朝时期是否有过与美洲的贸易往来？请给出详细说明。
"""

print("❌ 直接询问的提示:")
print(direct_prompt)
print("=" * 50)

response_direct = get_completion(direct_prompt)
print("模型回答:")
print(response_direct)


In [ ]:
# ✅ 正确示例：要求基于证据的分步分析
evidence_based_prompt = """
关于中国明朝时期是否有过与美洲的贸易往来这个问题，请按以下步骤分析：

第一步：时间线分析
- 明朝的时间范围（哪一年到哪一年）
- 欧洲人发现美洲的时间
- 这两个时间段是否有重叠

第二步：证据收集
- 列出你知道的相关历史事实和证据
- 明确指出哪些信息你确定，哪些不确定

第三步：逻辑推理
- 基于上述证据进行分析
- 考虑当时的技术条件、地理因素、政治环境

第四步：得出结论
- 给出你的答案，并说明可信度
- 如果证据不足，请明确说明

请按照这个结构来回答，不要跳过任何步骤。
"""

print("✅ 基于证据的分步提示:")
print(evidence_based_prompt)
print("=" * 50)

response_evidence = get_completion(evidence_based_prompt)
print("模型回答:")
print(response_evidence)


In [ ]:
# 💡 在这里编写你改进后的提示
# 可以参考上面的示例和模板

# 练习1：改进诺贝尔奖查询
improved_prompt_1 = """
# 在这里写你改进后的提示
"""

# 练习2：改进量子计算预测
improved_prompt_2 = """
# 在这里写你改进后的提示
"""

# 练习3：改进股价预测
improved_prompt_3 = """
# 在这里写你改进后的提示
"""

# 测试你的提示（取消注释来运行）
# print("测试改进后的提示1:")
# response1 = get_completion(improved_prompt_1)
# print(response1)
# print("\n" + "="*50 + "\n")

# print("测试改进后的提示2:")
# response2 = get_completion(improved_prompt_2)
# print(response2)
